# Corner plot with responsive histograms

The main code is in ../code/bkcorner and can be called from the notebook. Simply put the data in a Pandas DataFrame and choose which parameters you want to plot. By default, all parameters will be plotted.

In [1]:
import sys
sys.path.append("../code/")

import numpy as np
import pandas as pd
import bkcorner

In [2]:
df = pd.read_csv('../data/example_posterior.csv')
params=['log10Mbh','Thetai','Thetao','Gamma']

bkcorner.BKCorner(df, params=params, panel_width=150, notebook_url="http://localhost:8888")

Loading BokehJS ...

# CARAMEL input file plotter

This takes three of the four CARAMEL input files and plots the continuum and integrated emission line light curves as well as the emission line spectra. Hovering over the light curves will give information for the data points and the corresponding emission line spectrum will be highlighted in the right-hand panel.

A copy of this is saved in ../bkapps/caramel_file_viewer and can be run with "bokeh serve caramel_file_viewer" from the ../bkapps/ directory.

In [5]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row, column
from bokeh.models import CrosshairTool, WheelZoomTool, ResetTool, PanTool, BoxSelectTool, HoverTool, BoxZoomTool
from bokeh.models import ColumnDataSource, CustomJS, Div, DatePicker, FileInput

from base64 import b64decode

output_notebook()

Loading BokehJS ...

In [6]:
def modify_doc(doc):
    # First choose some default data to show
    spectra = np.loadtxt('../data/spectra.txt')
    cont = np.loadtxt('../data/continuum.txt')
    times = np.loadtxt('../data/times.txt')
    cont[:,0] /= 86400.
    times[:,0] /= 86400.
    cont[:,0] -= 2450000
    times[:,0] -= 2450000
    wave = spectra[0]
    spec = spectra[1::2]
    err = spectra[2::2]


    df_cont = pd.DataFrame(cont, columns=['time','flux','flux_err'])
    df_int = pd.DataFrame(
        np.vstack([
            times[:,0],
            np.sum(spec,axis=1),
            np.sqrt(np.sum(err**2,axis=1))
        ]).T,
        columns=['time','flux','flux_err']
    )

    # TODO: Temporary fix -- currently appending np.zeros() spectra to the end of the
    #       spectrum DataFrame. This is to account for the new uploaded files to have
    #       more spectra than the default. Currently maxed at 200.
    npad = 200
    pad = np.zeros((npad-len(spec),len(spec[0])))
    df_spec = pd.DataFrame(np.vstack([wave,spec,pad]).T, columns=['wave'] + [str(i) for i in range(npad)])  #, columns=['wave']+times[:,0].tolist())
    source_spec = ColumnDataSource(df_spec)
    df_cont = pd.DataFrame(cont, columns=['time','flux','flux_err'])
    source_cont = ColumnDataSource(df_cont)
    source_int = ColumnDataSource(df_int)

    # Create the figure panels
    # Continuum
    p1 = figure(
        width=400,height=150,
        x_axis_label='HJD - 2450000',
        y_axis_label='Continuum',
        tools=""
    )
    p1.scatter('time','flux', color='blue', source=source_cont, name='continuum')

    # Integrated emission line
    p2 = figure(
        width=400,height=150,
        x_axis_label='HJD - 2450000',
        y_axis_label='Emission line',
        x_range=p1.x_range,
        tools=""
    )
    p2.scatter('time','flux', color='orange', source=source_int, name='integrated')

    # Emission line spectra
    p3 = figure(
        width=300,height=300,
        x_axis_label='Wavelength (Ang)',
        y_axis_label='Flux',
    )

    for i in range(npad):
        p3.line('wave', str(i), source=source_spec, color='grey', width=0.5, alpha=0.5)
    # Create "highlighted" versions of each spectrum, but set visible=False
    line_highlighted = {}
    for i in range(npad):
        line_highlighted[i] = p3.line('wave', str(i), source=source_spec, color='orange', width=2, visible=False)

    # CustomJS callback for the hover tool
    callback_hover_js = CustomJS(args=dict(line=line_highlighted, source=source_int), code="""
        const data = source.data;
        const times = data.time;

        // Reset all previously highlighted lines
        for (var i=0; i<times.length; i++) {
            line[i].properties.visible.spec.value = false;
            line[i].change.emit();
        }

        // Get the x position of the cursor
        const geometry = cb_data.geometry;
        const xval = geometry['x'];

        // Find the closest of the plotted dates
        var closest;
        var distance = 1e9;  // Require maximum 1e9 day distance
        for (var i=0; i<times.length; i++) {
            if (Math.abs(xval - times[i]) < distance) {
                closest = i;
                distance = Math.abs(xval - times[i]);
            }
        }

        // Dynamically set the maximum distance to highlight
        // This should account for, various lengths of observing campaigns.
        const maxdist = (times[times.length-1] - times[0])/50;
        if (distance < maxdist) {
            line[closest].properties.visible.spec.value = true;
            line[closest].change.emit();
        }
        """)

    TOOLTIPS = [
        ('MJD', '@time'),
        ('Flux','@flux'),
        ('Error','@flux_err')
    ]
    TOOLS = [
        PanTool(),
        WheelZoomTool(dimensions='width'),
        CrosshairTool(dimensions='height'),
        HoverTool(
             tooltips=TOOLTIPS,
             mode='vline',
             #names=['continuum','integrated'],
             line_policy='none',
             callback=callback_hover_js
         )
    ]
    for tool in TOOLS:
        p1.add_tools(tool)
        p2.add_tools(tool)
    p1.toolbar.active_scroll = TOOLS[1]
    p2.toolbar.active_scroll = TOOLS[1]
    p1.toolbar_location = None
    p2.toolbar_location = None

    # Array to keep track of which files have been loaded
    new_files = np.array([0,0,0])
    def update_data():
        new_files = np.array([0,0,0])
        print("New files loaded")

        cont[:,0] /= 86400.
        times[:,0] /= 86400.
        cont[:,0] -= 2450000
        times[:,0] -= 2450000

        new_data_int = {'time': times[:,0], 'flux': np.sum(spec,axis=1), 'flux_err': np.sqrt(np.sum(err**2,axis=1))}
        new_data_cont = {'time': cont[:,0], 'flux': cont[:,1], 'flux_err': cont[:,2]}
        new_data_spec = {'wave': wave}

        for i in range(len(spec)):
            new_data_spec[str(i)] = spec[i]
        for i in range(len(spec),npad):
            new_data_spec[str(i)] = np.zeros(len(spec[0]))

        source_int.data = new_data_int
        source_cont.data = new_data_cont
        source_spec.data = new_data_spec

    def upload_spectra(attr, old, new):
        global wave, spec, err
        decoded = b64decode(new).decode('ascii')
        spectra = np.array([[float(item) for item in line.split(' ')] for line in decoded.splitlines()[1:]])
        wave = spectra[0]
        spec = spectra[1::2]
        err = spectra[2::2]

        new_files[0] = 1
        if np.prod(new_files) == 1:
            update_data()

    def upload_continuum(attr, old, new):
        global cont
        decoded = b64decode(new).decode('ascii')
        cont = np.array([[float(item) for item in line.split(' ')] for line in decoded.splitlines()])

        new_files[1] = 1
        if np.prod(new_files) == 1:
            update_data()

    def upload_times(attr, old, new):
        global times
        decoded = b64decode(new).decode('ascii')
        times = np.array([[float(item) for item in line.split(' ')] for line in decoded.splitlines()])

        new_files[2] = 1
        if np.prod(new_files) == 1:
            update_data()

    div_spec = Div(text="Spectrum:")
    div_time = Div(text="Times:")
    div_cont = Div(text="Continuum:")
    file_input_spec = FileInput(accept=".txt")
    file_input_spec.on_change('value', upload_spectra)
    file_input_time = FileInput(accept=".txt")
    file_input_time.on_change('value', upload_times)
    file_input_cont = FileInput(accept=".txt")
    file_input_cont.on_change('value', upload_continuum)

    # Set the layout with the sliders and plot
    layout = row(
        column(p1, p2),
        p3,
        column(div_spec, file_input_spec, div_time, file_input_time, div_cont, file_input_cont)
    )

    # add the layout to doc
    doc.add_root(layout)

# To display in the jupyter notebook, you need to provide the notebook_url
show(modify_doc, notebook_url="http://localhost:8888")

# Building the CARAMEL model

This creates a minimal version of the BLR model used in the CARAMEL modeling code. The edge-on and face-on views typically shown in papers are displayed along with the Gamma distribution from which the BLR cloud radii are drawn as well as the transfer function. The user can adjust some of the model parameters using the sliders and the four panels will respond.

In [7]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Slider, Button

output_notebook()

Loading BokehJS ...

In [8]:
def rotate(x, y, C, S):
    return C*x + S*y, -S*x + C*y

def buildModel(randoms, params):
    rand1, rand2, rand3, phi = randoms['rand1'], randoms['rand2'], randoms['rand3'], randoms['phi']
    beta, mu, F = params['beta'], params['mu'], params['F']
    thetao, thetai, gamma = params['thetao'], params['thetai'], params['gamma']
    kappa, Mbh = params['kappa'], 10**params['logMbh']
    
    alpha = beta**-2.0
    rmin = mu * F
    theta = (mu - rmin) / alpha
    
    # Compute the gamma function
    #  Using a shortcut here for computational purposes
    rmax = 500.
    xvals = np.linspace(0,rmax,500)
    
    # Compute the gamma distribution from 0 to rmax, then normalize
    gamma_distr = (xvals - rmin)**(alpha - 1.) * np.exp(-(xvals - rmin) / theta)
    gamma_distr = np.where(~np.isnan(gamma_distr), gamma_distr, 0.0)
    gamma_distr = np.where(gamma_distr > 0.0, gamma_distr, 0.0)
    gamma_distr = gamma_distr / np.sum(gamma_distr)
    
    # Compute the cdf
    cdf = [np.sum(gamma_distr[:i+1]) for i in range(len(gamma_distr))]
    
    # Take sorted distribution of values 0-1 to draw from cdf
    r = np.array([xvals[np.argmin(abs(cdf - rand))] for rand in rand1])

    # Determine the per-particle opening angles
    part1 = np.cos(thetao * np.pi / 180.)
    part2 = 1. - np.cos(thetao * np.pi / 180.)
    part3 = np.exp(np.log(rand2) * gamma)
    angle = np.arccos(part1 + part2 * part3)
    
    # Pre-calculate some sines and cosines
    sin1 = np.sin(angle)
    cos1 = np.cos(angle)
    sin2 = np.sin(rand3 * np.pi)
    cos2 = np.cos(rand3 * np.pi)
    sin3 = np.sin(0.5 * np.pi - thetai * np.pi / 180.)
    cos3 = np.cos(0.5 * np.pi - thetai * np.pi / 180.)
    
    sinPhi = np.sin(phi)
    cosPhi = np.cos(phi)

    ################################################
    # Set the positions
    # Turn radial distribution into a disk
    x = r * cosPhi
    y = r * sinPhi
    z = np.zeros(len(r))
    
    # Puff up by opening angle
    x, z = rotate(x, z, cos1, sin1)
    
    # Restore axi-symmetry
    x, y = rotate(x, y, cos2, sin2)
    
    # Rotate by inclination angle (+pi/2)
    x, z = rotate(x, z, cos3, sin3)

    ################################################
    # Compute the velocities
    G = 6.673e-11

    # Assume purely circular for now
    vr = np.zeros(len(r))
    vphi = np.sqrt(G * Mbh / r)  # * sin(theta)*exp(radial_sd_orbiting*n2[i][j]);
    
    # Convert to cartesian coordinates
    vx = vr * cosPhi - vphi * sinPhi    
    vy = vr * sinPhi + vphi * cosPhi
    vz = np.zeros(len(r))
    
    # Puff up by opening angle
    vx, vz = rotate(vx, vz, cos1, sin1)
    # Restore axi-symmetry
    vx, vy = rotate(vx, vy, cos2, sin2)
    # Rotate by inclination angle (+pi/2)
    vx, vz = rotate(vx, vz, cos3, sin3)
    # Convert vx to km/s
    vx *= 299792458. / 1000.
    
    ################################################
    # Compute the weights
    size = 0.5 + kappa * x/r
    size = np.where(np.isnan(size),0.0,size)
    size *= 1.0 * len(size) / np.sum(size)
    
    ################################################
    # Compute the lags
    lags = r - x
    
    return x, y, z, vx, lags, size, xvals, gamma_distr

In [9]:
def modify_doc(doc):
    Ndata = 1000
    rand1 = np.linspace(0,1,Ndata)
    phi = np.random.uniform(0,2.*np.pi,Ndata)
    rand2 = np.random.uniform(-1,1,Ndata)
    rand3 = np.random.uniform(-1,1,Ndata)
    randoms = {'rand1':rand1, 'rand2':rand2, 'rand3':rand3, 'phi':phi}
    
    # Set the starting data
    start = {
        'thetai': 40.,
        'thetao': 30.,
        'kappa': -0.4,
        'beta': 0.85,
        'mu': 19.5,
        'F': 0.29,
        'logMbh': 7.9,
        'gamma': 3.0,
    }
    
    x, y, z, vx, lags, size, gamma_x, gamma_y = buildModel(randoms, start)
    source_model = ColumnDataSource(data=dict(
        x=x, y=y, z=z, size=size, lags=lags, vx=vx,
        rand1=rand1,rand2=rand2,rand3=rand3,phi=phi
    ))
    source_gamma = ColumnDataSource(data=dict(gamma_x=gamma_x,gamma_y=gamma_y))

    # Create the figure panels
    p_edge = figure(x_range=(-50, 50), y_range=(-50, 50), plot_width=300, plot_height=300, toolbar_location=None)
    p_face = figure(x_range=(-50, 50), y_range=(-50, 50), plot_width=300, plot_height=300, toolbar_location=None)
    p_tf = figure(x_range=(-15000, 15000), y_range=(0, 100), plot_width=300, plot_height=300, toolbar_location=None)
    p_gamma = figure(x_range=(0, 100), plot_width=300, plot_height=300, toolbar_location=None)

    p_edge.xaxis.axis_label = "x (light days)"
    p_edge.yaxis.axis_label = "z (light days)"
    p_face.xaxis.axis_label = "y (light days)"
    p_face.yaxis.axis_label = "z (light days)"
    p_tf.xaxis.axis_label = "Velocity (km/s)"
    p_tf.yaxis.axis_label = "Lag (light days)"
    p_gamma.xaxis.axis_label = "r (light days)"
    p_gamma.yaxis.axis_label = "P(r)"

    p_edge.scatter('x', 'z', size='size', source=source_model)
    p_face.scatter('y', 'z', size='size', source=source_model)
    p_tf.scatter('vx', 'lags', size='size', source=source_model)
    p_gamma.scatter('gamma_x', 'gamma_y', source=source_gamma)
    
    # Create the slider widgets
    sliders = {
        "thetai": Slider(start=0.0, end=90., value=start['thetai'], step=1, title="Inclination angle (deg)"),
        "thetao": Slider(start=0.0, end=90., value=start['thetao'], step=1, title="Opening angle (deg)"),
        "gamma": Slider(start=1.0, end=5.0, value=start['gamma'], step=0.1, title="Gamma"),
        "kappa": Slider(start=-0.5, end=0.5, value=start['kappa'], step=0.05, title="Kappa"),
        "beta": Slider(start=0.1, end=2.0, value=start['beta'], step=0.05, title="Beta"),
        "mu": Slider(start=7.5, end=72.5, value=start['mu'], step=1, title="Mu (light days)"),
        "F": Slider(start=0.125, end=0.825, value=start['F'], step=0.01, title="F"),
        "logMbh": Slider(start=6.5, end=8.5, value=start['logMbh'], step=0.1, title="log10(Mbh/Msun)"),
    }
    reset_button = Button(label='Reset')

    # Set the code to update the data when the sliders are change
    def callback_sliders(attr, old, new):
        params = {key: sliders[key].value for key in sliders.keys()}
        x, y, z, vx, lags, size, gamma_x, gamma_y = buildModel(randoms, params)
        new_model_data = dict(x=x,y=y,z=z,vx=vx,lags=lags,size=size)
        new_gamma_data = dict(gamma_x=gamma_x,gamma_y=gamma_y)
        source_model.data = new_model_data
        source_gamma.data = new_gamma_data
    
    def callback_reset():
        for key in sliders.keys():
            sliders[key].value = start[key]
        x, y, z, vx, lags, size, gamma_x, gamma_y = buildModel(randoms, start)
        new_model_data = dict(x=x,y=y,z=z,vx=vx,lags=lags,size=size)
        new_gamma_data = dict(gamma_x=gamma_x,gamma_y=gamma_y)
        source_model.data = new_model_data
        source_gamma.data = new_gamma_data

    for key in sliders.keys():
        sliders[key].on_change("value", callback_sliders)
    reset_button.on_click(callback_reset)
    # Set the layout with the sliders and plot
    layout = row(
        column(*[sliders[key] for key in sliders.keys()], reset_button),
        column(
            row(p_edge,p_face),
            row(p_gamma, p_tf)
        )
    )

    # add the layout to curdoc
    doc.add_root(layout)

show(modify_doc, notebook_url="http://localhost:8888")

/anaconda3/envs/bokeh/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power
/anaconda3/envs/bokeh/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in log
/anaconda3/envs/bokeh/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in true_divide
/anaconda3/envs/bokeh/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in add
  
/anaconda3/envs/bokeh/lib/python3.7/site-packages/ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in true_divide
